In [ ]:
import numpy as np
import cv2
import math


def hough_lines():
    src = cv2.imread('../data/img/mille_girl.png', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    edge = cv2.Canny(src, 50, 150)
    lines = cv2.HoughLines(edge, 1, math.pi / 180, 250)

    dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

    if lines is not None:
        for i in range(lines.shape[0]):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            cos_t = math.cos(theta)
            sin_t = math.sin(theta)
            x0, y0 = rho * cos_t, rho * sin_t
            alpha = 1000
            pt1 = (int(x0 - alpha * sin_t), int(y0 + alpha * cos_t))
            pt2 = (int(x0 + alpha * sin_t), int(y0 - alpha * cos_t))
            cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


def hough_line_segments():
    src = cv2.imread('../data/img/mille_girl.png', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    edge = cv2.Canny(src, 50, 150)
    lines = cv2.HoughLinesP(edge, 1, math.pi / 180, 160, minLineLength=50, maxLineGap=5)

    dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

    if lines is not None:
        for i in range(lines.shape[0]):
            pt1 = (lines[i][0][0], lines[i][0][1])
            pt2 = (lines[i][0][2], lines[i][0][3])
            cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


def hough_circles():
    src = cv2.imread('coins.png', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    blurred = cv2.blur(src, (3, 3))
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 50,
                              param1=150, param2=30)

    dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

    if circles is not None:
        for i in range(circles.shape[1]):
            cx, cy, radius = circles[0][i]
            cv2.circle(dst, (cx, cy), int(radius), (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    hough_lines()
    hough_line_segments()
    hough_circles()


In [8]:
import cv2
import numpy as np

# 이미지 읽기
img = cv2.imread('../data/img/coins.png')
rows, cols = img.shape[:2]
cv2.imshow('original', img)


# 동전 표면을 흐릿하게 피라미드평균시프트 적용
mean = cv2.pyrMeanShiftFiltering(img, 20, 50)
cv2.imshow('mean', mean)
# 바이너리 이미지 변환
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3,3), 0)

_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)
# 거리 변환
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 3)
# 거리 값을 0 ~255로 변환
dst = ( dst / (dst.max() - dst.min()) * 255 ).astype(np.uint8)
cv2.imshow('dst', dst)

# 거리 변환결과에서 로칼 최대 값 구하기
## 팽창 적용(동전 크기 정도의 구조화 요소 필요),
localMx = cv2.dilate(dst, np.ones((50,50), np.uint8))
## 로칼 최대 값 저장 할 배열 생성
lm = np.zeros((rows, cols), np.uint8)
## 팽창 적용전 이미지와 같은 픽셀이 로컬 최대 값이므로 255로 설정
lm[(localMx==dst) & (dst != 0)] = 255
cv2.imshow('localMx', lm)

# 로컬 최대값으로 색 채우기
## 로컬 최대 값이 있는 좌표 구하기
seeds = np.where(lm ==255)
seed = np.stack( (seeds[1], seeds[0]), axis=-1)
## 색 채우기를 위한 채우기 마스크 생성
fill_mask = np.zeros((rows+2, cols+2), np.uint8)
for x,y in seed:
    ## 로칼 최대값을 시드로해서 평균 시프트 영상에 색채우기 
    ret = cv2.floodFill(mean, fill_mask, (x,y), (255,255,255), \
                                            (10,10,10), (10,10,10))
cv2.imshow('floodFill', mean)

# 색 채우기 적용한 영상에 다시 거리 변환 적용
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0)

ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
dst = ( (dst / (dst.max() - dst.min())) * 255 ).astype(np.uint8)
cv2.imshow('dst2', dst)

# 거리 변환 결과값의 절반 이상을 차지한 영역은 확실한 전경으로 설정
ret, sure_fg = cv2.threshold(dst, 0.5*dst.max(), 255,0)
cv2.imshow('sure_fg', sure_fg)

# 거리 변환 결과를 반전해서 확실한 배경 찾기
_, bg_th = cv2.threshold(dst, 0.3*dst.max(),  255, cv2.THRESH_BINARY_INV)
bg_dst = cv2.distanceTransform(bg_th, cv2.DIST_L2, 5)
bg_dst = ( (bg_dst / (bg_dst.max() - bg_dst.min())) * 255 ).astype(np.uint8)
ret, sure_bg = cv2.threshold(bg_dst, 0.3*bg_dst.max(), 255,cv2.THRESH_BINARY)
cv2.imshow('sure_bg', sure_bg)


# 불확실한 영역 설정 : 확실한 배경을 반전해서 확실한 전경을 빼기
ret, inv_sure_bg = cv2.threshold(sure_bg, 127, 255,cv2.THRESH_BINARY_INV)
unkown = cv2.subtract(inv_sure_bg, sure_fg)
cv2.imshow('unkown', unkown)

# 연결된 요소 레이블링
_, markers = cv2.connectedComponents(sure_fg)

# 레이블링을 1씩 증가 시키고 0번 레이블 알 수 없는 영역을 0번 레이블로 설정
markers = markers+1
markers[unkown ==255] = 0
print("워터쉐드 전:", np.unique(markers))
colors = []
marker_show = np.zeros_like(img)
for mid in np.unique(markers): # 선택한 마커 아이디 갯수 만큼 반복
    color = [int(j) for j in np.random.randint(0,255, 3)]
    colors.append((mid, color))
    marker_show[markers==mid] = color
    coords = np.where(markers==mid)
    x, y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, \
                                                             2, (255,255,255))
cv2.imshow('before', marker_show)

# 레이블링이 완성된 마커로 워터 쉐드 적용
markers = cv2.watershed(img, markers)
print("워터쉐드 후:", np.unique(markers))

for mid, color in colors: # 선택한 마커 아이디 갯수 만큼 반복
    marker_show[markers==mid] = color
    coords = np.where(markers==mid)
    if coords[0].size <= 0 : 
        continue
    x, y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, \
                                                             2, (255,255,255))
marker_show[markers==-1] = (0,255,0)
cv2.imshow('watershed marker', marker_show)

img[markers==-1] = (0,255,0)
cv2.imshow('watershed', img)

# 동전 추출을 위한 마스킹 생성
mask = np.zeros((rows, cols), np.uint8)
# 배경 마스크 생성
mask[markers!=1] = 255
# 배경 지우기
nobg = cv2.bitwise_and(img, img, mask=mask)
# 동전만 있는 라벨 생성 (배경(1), 경계(-1) 없는)
coin_label = [l for l in np.unique(markers) if (l != 1 and l !=-1)]
# 동전 라벨 순회 하면서 동전 영역만 추출
for i, label in enumerate(coin_label):
    mask[:,:] = 0
    # 해당 동전 추출 마스크 생성
    mask[markers ==label] = 255
    # 동전 영역만 마스크로 추출
    coins = cv2.bitwise_and(img, img, mask=mask)
    # 동전 하나만 있는 곳에서 최외곽 컨투어 추출
    contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,\
                                         cv2.CHAIN_APPROX_NONE)[-2:]
    # 동전을 감싸는 사각형 좌표
    x,y,w,h = cv2.boundingRect(contour[0])
    # 동전 영역만 추출해서 출력
    coin = coins[y:y+h, x:x+w]
    cv2.imshow('coin%d'%(i+1), coin)
    cv2.imwrite('../img/coin_test/coin%d.jpg'%(i+1), coin)
cv2.waitKey()
cv2.destroyAllWindows()

워터쉐드 전: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97]
워터쉐드 후: [-1  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97]


In [3]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('../data/img/coins.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur to reduce noise and improve contour detection
blurred = cv2.GaussianBlur(gray, (11, 11), 0)

# Apply a binary threshold to get a binary image
_, binary = cv2.threshold(blurred, 120, 255, cv2.THRESH_BINARY_INV)

# Find contours in the binary image
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image
cv2.drawContours(img, contours, -1, (0, 255, 0), 2)

# Display the number of coins found
num_coins = len(contours)
print(f"Number of coins detected: {num_coins}")

# Display the images
cv2.imshow('Original Image', img)
cv2.imshow('Binary Image', binary)

cv2.waitKey(0)
cv2.destroyAllWindows()

Number of coins detected: 244


In [4]:
import cv2
import numpy as np

# 이미지 읽기
img = cv2.imread('../data/img/coins.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 노이즈 제거를 위해 가우시안 블러 적용
blurred = cv2.GaussianBlur(gray, (11, 11), 0)

# 적응형 이진화 적용
adaptive_thresh = cv2.adaptiveThreshold(
    blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV, 11, 2)

# 컨투어 찾기
contours, _ = cv2.findContours(adaptive_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 원본 이미지에 컨투어 그리기
cv2.drawContours(img, contours, -1, (0, 255, 0), 2)

# 동전 개수 출력
num_coins = len(contours)
print(f"Number of coins detected: {num_coins}")

# 이미지 출력
cv2.imshow('Original Image with Contours', img)
cv2.imshow('Adaptive Threshold', adaptive_thresh)

cv2.waitKey(0)
cv2.destroyAllWindows()


Number of coins detected: 809


In [5]:
import cv2
import numpy as np

# 이미지 읽기
img = cv2.imread('../data/img/coins.png')
rows, cols = img.shape[:2]
cv2.imshow('original', img)

# 동전 표면을 흐릿하게 피라미드 평균 시프트 적용
mean = cv2.pyrMeanShiftFiltering(img, 20, 50)
cv2.imshow('mean', mean)

# 바이너리 이미지 변환
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)

# 거리 변환
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 3)
dst = (dst / (dst.max() - dst.min()) * 255).astype(np.uint8)
cv2.imshow('dst', dst)

# 거리 변환 결과에서 로컬 최대값 구하기
localMx = cv2.dilate(dst, np.ones((50, 50), np.uint8))
lm = np.zeros((rows, cols), np.uint8)
lm[(localMx == dst) & (dst != 0)] = 255
cv2.imshow('localMx', lm)

# 로컬 최대값으로 색 채우기
seeds = np.where(lm == 255)
seed = np.stack((seeds[1], seeds[0]), axis=-1)
fill_mask = np.zeros((rows + 2, cols + 2), np.uint8)
for x, y in seed:
    cv2.floodFill(mean, fill_mask, (x, y), (255, 255, 255), (10, 10, 10), (10, 10, 10))

cv2.imshow('floodFill', mean)

# 색 채우기 적용한 영상에 다시 거리 변환 적용
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
dst = ((dst / (dst.max() - dst.min())) * 255).astype(np.uint8)
cv2.imshow('dst2', dst)

# 거리 변환 결과값의 절반 이상을 차지한 영역은 확실한 전경으로 설정
ret, sure_fg = cv2.threshold(dst, 0.5 * dst.max(), 255, 0)
cv2.imshow('sure_fg', sure_fg)

# 거리 변환 결과를 반전해서 확실한 배경 찾기
_, bg_th = cv2.threshold(dst, 0.3 * dst.max(), 255, cv2.THRESH_BINARY_INV)
bg_dst = cv2.distanceTransform(bg_th, cv2.DIST_L2, 5)
bg_dst = ((bg_dst / (bg_dst.max() - bg_dst.min())) * 255).astype(np.uint8)
ret, sure_bg = cv2.threshold(bg_dst, 0.3 * bg_dst.max(), 255, cv2.THRESH_BINARY)
cv2.imshow('sure_bg', sure_bg)

# 불확실한 영역 설정: 확실한 배경을 반전해서 확실한 전경을 빼기
ret, inv_sure_bg = cv2.threshold(sure_bg, 127, 255, cv2.THRESH_BINARY_INV)
unkown = cv2.subtract(inv_sure_bg, sure_fg)
cv2.imshow('unkown', unkown)

# 연결된 요소 레이블링
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unkown == 255] = 0

# Watershed 적용
markers = cv2.watershed(img, markers)
img[markers == -1] = [0, 255, 0]  # 경계를 빨간색으로 표시
cv2.imshow('watershed', img)

# 동전 추출을 위한 마스킹 생성
mask = np.zeros((rows, cols), np.uint8)
mask[markers != 1] = 255
nobg = cv2.bitwise_and(img, img, mask=mask)

# 동전 라벨 추출 및 저장
coin_label = [l for l in np.unique(markers) if (l != 1 and l != -1)]
for i, label in enumerate(coin_label):
    mask[:, :] = 0
    mask[markers == label] = 255
    coins = cv2.bitwise_and(img, img, mask=mask)
    contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    for cnt in contour:
        # 동전을 감싸는 사각형 좌표
        x, y, w, h = cv2.boundingRect(cnt)
        # 동전 영역만 추출
        coin = coins[y:y+h, x:x+w]
        
        # 원형 검출
        gray_coin = cv2.cvtColor(coin, cv2.COLOR_BGR2GRAY)
        circles = cv2.HoughCircles(
            gray_coin, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20, 
            param1=50, param2=30, minRadius=10, maxRadius=100
        )
        
        # 동전이 원형인 경우 표시
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            for (x_center, y_center, radius) in circles:
                cv2.circle(coin, (x_center, y_center), radius, (0, 255, 0), 2)
                cv2.rectangle(coin, (x_center - 5, y_center - 5), (x_center + 5, y_center + 5), (0, 128, 255), -1)
            cv2.putText(coin, 'Coin %d' % (i + 1), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        cv2.imshow(f'coin{i+1}', coin)
        cv2.imwrite(f'../img/coin_test/coin{i+1}.jpg', coin) 

cv2.waitKey(0)
cv2.destroyAllWindows()


In [6]:
import cv2
import numpy as np

# 이미지 읽기
img = cv2.imread('../data/img/coins.png')
rows, cols = img.shape[:2]
cv2.imshow('original', img)

# 동전 표면을 흐릿하게 피라미드 평균 시프트 적용
mean = cv2.pyrMeanShiftFiltering(img, 20, 50)
cv2.imshow('mean', mean)

# 바이너리 이미지 변환
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)

# 거리 변환
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 3)
dst = (dst / (dst.max() - dst.min()) * 255).astype(np.uint8)
cv2.imshow('dst', dst)

# 거리 변환 결과에서 로컬 최대값 구하기
localMx = cv2.dilate(dst, np.ones((50, 50), np.uint8))
lm = np.zeros((rows, cols), np.uint8)
lm[(localMx == dst) & (dst != 0)] = 255
cv2.imshow('localMx', lm)

# 로컬 최대값으로 색 채우기
seeds = np.where(lm == 255)
seed = np.stack((seeds[1], seeds[0]), axis=-1)
fill_mask = np.zeros((rows + 2, cols + 2), np.uint8)
for x, y in seed:
    cv2.floodFill(mean, fill_mask, (x, y), (255, 255, 255), (10, 10, 10), (10, 10, 10))

cv2.imshow('floodFill', mean)

# 색 채우기 적용한 영상에 다시 거리 변환 적용
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
dst = ((dst / (dst.max() - dst.min())) * 255).astype(np.uint8)
cv2.imshow('dst2', dst)

# 거리 변환 결과값의 절반 이상을 차지한 영역은 확실한 전경으로 설정
ret, sure_fg = cv2.threshold(dst, 0.5 * dst.max(), 255, 0)
cv2.imshow('sure_fg', sure_fg)

# 거리 변환 결과를 반전해서 확실한 배경 찾기
_, bg_th = cv2.threshold(dst, 0.3 * dst.max(), 255, cv2.THRESH_BINARY_INV)
bg_dst = cv2.distanceTransform(bg_th, cv2.DIST_L2, 5)
bg_dst = ((bg_dst / (bg_dst.max() - bg_dst.min())) * 255).astype(np.uint8)
ret, sure_bg = cv2.threshold(bg_dst, 0.3 * bg_dst.max(), 255, cv2.THRESH_BINARY)
cv2.imshow('sure_bg', sure_bg)

# 불확실한 영역 설정: 확실한 배경을 반전해서 확실한 전경을 빼기
ret, inv_sure_bg = cv2.threshold(sure_bg, 127, 255, cv2.THRESH_BINARY_INV)
unkown = cv2.subtract(inv_sure_bg, sure_fg)
cv2.imshow('unkown', unkown)

# 연결된 요소 레이블링
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unkown == 255] = 0

# Watershed 적용
markers = cv2.watershed(img, markers)
img[markers == -1] = [0, 255, 0]  # 경계를 초록색으로 표시
cv2.imshow('watershed', img)

# 동전 추출을 위한 마스킹 생성
mask = np.zeros((rows, cols), np.uint8)
mask[markers != 1] = 255
nobg = cv2.bitwise_and(img, img, mask=mask)

# 동전 라벨 추출 및 저장
coin_label = [l for l in np.unique(markers) if (l != 1 and l != -1)]

# 총 동전 개수
total_coins = 0

for i, label in enumerate(coin_label):
    mask[:, :] = 0
    mask[markers == label] = 255
    coins = cv2.bitwise_and(img, img, mask=mask)
    contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    for cnt in contour:
        # 동전을 감싸는 사각형 좌표
        x, y, w, h = cv2.boundingRect(cnt)
        # 동전 영역만 추출
        coin = coins[y:y+h, x:x+w]
        
        # 원형 검출
        gray_coin = cv2.cvtColor(coin, cv2.COLOR_BGR2GRAY)
        circles = cv2.HoughCircles(
            gray_coin, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20, 
            param1=100, param2=30, minRadius=10, maxRadius=100
        )
        
        # 동전이 원형인 경우 표시
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            total_coins += len(circles)  # 검출된 원의 수를 동전 개수에 추가
            for (x_center, y_center, radius) in circles:
                cv2.circle(coin, (x_center, y_center), radius, (0, 255, 0), 2)
                cv2.rectangle(coin, (x_center - 5, y_center - 5), (x_center + 5, y_center + 5), (0, 128, 255), -1)
            cv2.putText(coin, 'Coin %d' % (i + 1), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        cv2.imshow(f'coin{i+1}', coin)
        cv2.imwrite(f'../img/coin_test/coin{i+1}.jpg', coin) 

# 동전의 총 개수 출력
print(f'Total number of coins detected: {total_coins}')

cv2.waitKey(0)
cv2.destroyAllWindows()


Total number of coins detected: 348


In [7]:
import cv2
import numpy as np
 
src = cv2.imread('../data/img/coins.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit() 

blurred = cv2.GaussianBlur(src, (0, 0), 10)  
alpha = 1.0
dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0) 
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()


In [43]:
import cv2
import numpy as np
import sys

# 이미지 읽기
src = cv2.imread('../data/img/coin3.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 가우시안 블러 적용
blurred = cv2.GaussianBlur(src, (0, 0), 10)  

# 이미지 결합 (샤프닝 효과)
alpha = 1.0
dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0) 
cv2.imshow('dst', dst)

# 이진화
_, thresh = cv2.threshold(dst, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)

# 거리 변환
dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
cv2.normalize(dist_transform, dist_transform, 0, 255, cv2.NORM_MINMAX)
dist_transform = np.uint8(dist_transform)
cv2.imshow('dist_transform', dist_transform)

# 거리 변환 결과에서 전경과 배경을 구분
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
cv2.imshow('sure_fg', sure_fg)

# 거리 변환 결과의 역전
sure_bg = cv2.dilate(thresh, np.ones((3, 3), np.uint8))
sure_bg = cv2.subtract(sure_bg, thresh)
cv2.imshow('sure_bg', sure_bg)

# 불확실한 영역 구하기
unknown = cv2.subtract(sure_bg, sure_fg)
cv2.imshow('unknown', unknown)

# 연결된 구성 요소 레이블링
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Watershed 알고리즘 적용
markers = cv2.watershed(cv2.cvtColor(src, cv2.COLOR_GRAY2BGR), markers)
cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)[markers == -1] = [0, 255, 0]

# 윤곽선 찾기
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 윤곽선에서 원형 형태만 필터링하는 함수
def is_circular(contour, min_circularity=0.7):
    """윤곽선이 원형에 가까운지 확인"""
    if len(contour) == 0:
        return False
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    # 윤곽선의 원형성 계산
    if perimeter == 0:
        return False
    circularity = 4 * np.pi * area / (perimeter * perimeter)
    
    return circularity > min_circularity

# 원형 윤곽선 필터링
circular_contours = [cnt for cnt in contours if is_circular(cnt)]

# 원형 윤곽선 개수 출력
num_coins = len(circular_contours)
print(f'Total number of coins detected: {num_coins}')

# 윤곽선 그리기
output = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.drawContours(output, circular_contours, -1, (0, 255, 0), 2)
cv2.imshow('circular contours', output)

cv2.waitKey(0)
cv2.destroyAllWindows()


Total number of coins detected: 3


In [44]:
import cv2
import numpy as np
import sys

# 이미지 읽기
src = cv2.imread('../data/img/coin3.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 가우시안 블러 적용
blurred = cv2.GaussianBlur(src, (0, 0), 10)  

# 이미지 결합 (샤프닝 효과)
alpha = 1.0
dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0) 
cv2.imshow('dst', dst)

# 이진화
_, thresh = cv2.threshold(dst, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)

# 거리 변환
dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
cv2.normalize(dist_transform, dist_transform, 0, 255, cv2.NORM_MINMAX)
dist_transform = np.uint8(dist_transform)
cv2.imshow('dist_transform', dist_transform)

# 거리 변환 결과에서 전경과 배경을 구분
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
cv2.imshow('sure_fg', sure_fg)

# 거리 변환 결과의 역전
sure_bg = cv2.dilate(thresh, np.ones((3, 3), np.uint8))
sure_bg = cv2.subtract(sure_bg, thresh)
cv2.imshow('sure_bg', sure_bg)

# 불확실한 영역 구하기
unknown = cv2.subtract(sure_bg, sure_fg)
cv2.imshow('unknown', unknown)

# 연결된 구성 요소 레이블링
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Watershed 알고리즘 적용
markers = cv2.watershed(cv2.cvtColor(src, cv2.COLOR_GRAY2BGR), markers)
src[markers == -1] = 0  # 경계선은 0으로 설정
cv2.imshow('watershed', src)

# 윤곽선 찾기
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 윤곽선에서 원형 형태만 필터링하는 함수
def is_circular(contour, min_circularity=0.7):
    """윤곽선이 원형에 가까운지 확인"""
    if len(contour) == 0:
        return False
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    # 윤곽선의 원형성 계산
    if perimeter == 0:
        return False
    circularity = 4 * np.pi * area / (perimeter * perimeter)
    
    return circularity > min_circularity

# 원형 윤곽선 필터링
circular_contours = [cnt for cnt in contours if is_circular(cnt)]

# 원형 윤곽선 개수 출력
num_coins = len(circular_contours)
print(f'Total number of coins detected: {num_coins}')

# 윤곽선 그리기
output = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.drawContours(output, circular_contours, -1, (0, 255, 0), 2)
cv2.imshow('circular contours', output)

cv2.waitKey(0)
cv2.destroyAllWindows()


Total number of coins detected: 3


In [49]:


import cv2
import numpy as np
import sys

# 이미지 읽기
src = cv2.imread('../data/img/coin3.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 가우시안 블러 적용
blurred = cv2.GaussianBlur(src, (0, 0), 10)  

# 이미지 결합 (샤프닝 효과)
alpha = 1.0
dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0) 
cv2.imshow('dst', dst)

# 이진화
_, thresh = cv2.threshold(dst, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)

# 거리 변환
dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
cv2.normalize(dist_transform, dist_transform, 0, 255, cv2.NORM_MINMAX)
dist_transform = np.uint8(dist_transform)
cv2.imshow('dist_transform', dist_transform)

# 거리 변환 결과에서 전경과 배경을 구분
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
cv2.imshow('sure_fg', sure_fg)

# 거리 변환 결과의 역전
sure_bg = cv2.dilate(thresh, np.ones((3, 3), np.uint8))
sure_bg = cv2.subtract(sure_bg, thresh)
cv2.imshow('sure_bg', sure_bg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
import cv2
import numpy as np
import sys

# 이미지 읽기
src = cv2.imread('../data/img/coin3.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 가우시안 블러 적용
blurred = cv2.GaussianBlur(src, (0, 0), 10)

# 이미지 결합 (샤프닝 효과)
alpha = 1.0
dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0)

# 이진화
_, thresh = cv2.threshold(dst, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 거리 변환
dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
cv2.normalize(dist_transform, dist_transform, 0, 255, cv2.NORM_MINMAX)
dist_transform = np.uint8(dist_transform)

# 거리 변환 결과에서 전경과 배경을 구분
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# 거리 변환 결과의 역전
sure_bg = cv2.dilate(thresh, np.ones((3, 3), np.uint8))
sure_bg = cv2.subtract(sure_bg, thresh)

# 윤곽선 찾기
contours, _ = cv2.findContours(sure_bg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 원 그리기
output = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
for contour in contours:
    if cv2.contourArea(contour) > 100:  # 작은 노이즈 필터링 (조정 가능)
        # 최소 외접원 계산
        (x, y), radius = cv2.minEnclosingCircle(contour)
        center = (int(x), int(y))
        radius = int(radius)
        
        # 원의 음영 처리
        cv2.circle(output, center, radius, (0, 255, 0), 2)  # 원의 외곽을 녹색으로 그리기
        cv2.circle(output, center, radius // 3, (0, 255, 0), 1)  # 원의 중앙 부분을 작게 그리기

cv2.imshow('sure_bg with circles', output)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [53]:
import cv2
import numpy as np
import sys

def find_circles_from_contours(contours, min_area=100, max_distance=10):
    """윤곽선에서 원호를 찾아 원으로 변환"""
    circles = []
    for contour in contours:
        if cv2.contourArea(contour) > min_area:
            # 최소 외접원 계산
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)
            circles.append((center, radius))
    
    return circles

def draw_circles(output, circles, color=(0, 255, 0), thickness=2):
    """이미지에 원을 그리기"""
    for (center, radius) in circles:
        cv2.circle(output, center, radius, color, thickness)

def find_and_draw_nearby_circles(image, circles, max_distance=10):
    """주어진 원에서 max_distance 이내의 원을 찾아 그리기"""
    output = image.copy()
    for (center, radius) in circles:
        x, y = center
        for dy in range(-max_distance, max_distance+1):
            for dx in range(-max_distance, max_distance+1):
                nx, ny = x + dx, y + dy
                if 0 <= nx < image.shape[1] and 0 <= ny < image.shape[0]:
                    if np.linalg.norm([dx, dy]) <= max_distance:
                        if cv2.pointPolygonTest(contours[0], (nx, ny), True) >= 0:
                            cv2.circle(output, (int(nx), int(ny)), 2, (0, 0, 255), -1)
    return output

# 이미지 읽기
src = cv2.imread('../data/img/coin3.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 가우시안 블러 적용
blurred = cv2.GaussianBlur(src, (0, 0), 10)

# 이미지 결합 (샤프닝 효과)
alpha = 1.0
dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0)

# 이진화
_, thresh = cv2.threshold(dst, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 거리 변환
dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
cv2.normalize(dist_transform, dist_transform, 0, 255, cv2.NORM_MINMAX)
dist_transform = np.uint8(dist_transform)

# 거리 변환 결과에서 전경과 배경을 구분
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# 거리 변환 결과의 역전
sure_bg = cv2.dilate(thresh, np.ones((3, 3), np.uint8))
sure_bg = cv2.subtract(sure_bg, thresh)
 
cv2.imshow('Detected Circles and Nearby', sure_bg)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [54]:
import cv2
import numpy as np
import sys

def find_circles_from_contours(contours, min_area=100):
    """윤곽선에서 원호를 찾아 원으로 변환"""
    circles = []
    for contour in contours:
        if cv2.contourArea(contour) > min_area:
            # 최소 외접원 계산
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)
            circles.append((center, radius))
    
    return circles

def draw_circles(output, circles, color=(0, 255, 0), thickness=2):
    """이미지에 원을 그리기"""
    for (center, radius) in circles:
        cv2.circle(output, center, radius, color, thickness)

# 이미지 읽기
src = cv2.imread('../data/img/coin3.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 가우시안 블러 적용
blurred = cv2.GaussianBlur(src, (0, 0), 10)

# 이미지 결합 (샤프닝 효과)
alpha = 1.0
dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0)

# 이진화
_, thresh = cv2.threshold(dst, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 거리 변환
dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
cv2.normalize(dist_transform, dist_transform, 0, 255, cv2.NORM_MINMAX)
dist_transform = np.uint8(dist_transform)

# 거리 변환 결과에서 전경과 배경을 구분
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# 거리 변환 결과의 역전
sure_bg = cv2.dilate(thresh, np.ones((3, 3), np.uint8))
sure_bg = cv2.subtract(sure_bg, thresh)

# 윤곽선 찾기
contours, _ = cv2.findContours(sure_bg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 윤곽선에서 원호를 추출하여 원으로 변환
circles = find_circles_from_contours(contours)

# 원 그리기
output = cv2.cvtColor(sure_bg, cv2.COLOR_GRAY2BGR)
draw_circles(output, circles)

cv2.imshow('Detected Circles', sure_bg)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
import sys

def find_and_draw_circles_from_distance_transform(dist_transform, max_distance=10):
    """거리 변환 결과에서 원을 그립니다."""
    # 거리 변환 결과를 색상으로 변환
    colored_dist_transform = cv2.applyColorMap(dist_transform, cv2.COLORMAP_JET)
    
    # 원을 그릴 이미지 생성
    output = cv2.cvtColor(dist_transform, cv2.COLOR_GRAY2BGR)
    
    # 거리 변환 값에 따라 원을 그립니다
    for y in range(dist_transform.shape[0]):
        for x in range(dist_transform.shape[1]):
            radius = int(dist_transform[y, x])
            if radius > max_distance:
                center = (x, y)
                cv2.circle(output, center, radius, (0, 255, 0), 2)
                
    return output

# 이미지 읽기
src = cv2.imread('../data/img/coins.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 가우시안 블러 적용
blurred = cv2.GaussianBlur(src, (0, 0), 10)

# 이미지 결합 (샤프닝 효과)
alpha = 1.0
dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0)

# 이진화
_, thresh = cv2.threshold(dst, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 거리 변환
dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
cv2.normalize(dist_transform, dist_transform, 0, 255, cv2.NORM_MINMAX)
dist_transform = np.uint8(dist_transform)

# 원 그리기
output = find_and_draw_circles_from_distance_transform(dist_transform)

cv2.imshow('Detected Circles', output)
cv2.waitKey(0)
cv2.destroyAllWindows()
